In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# 데이터 로드
train_data = pd.read_excel('train_data.xlsx')
train_data = train_data.drop(['SP'], axis=1)
train_data = pd.get_dummies(train_data, columns=['AD_NO', 'LIV_ADD', 'RES_ADD', 'ADD_YN'])

In [3]:
# 입력 (X) / 출력 (Y) 변수 분리
X = train_data.drop('결과값(연체회차)', axis=1)
y = train_data['결과값(연체회차)']

In [13]:
# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

features = X.columns.tolist()

In [5]:
# 특성 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# 랜덤포레스트 학습
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
# 랜덤포레스트 예측
rf_train_pred = rf.predict(X_train)
rf_test_pred = rf.predict(X_test)

In [8]:
# 랜덤포레스트 정확도 계산
rf_train_accuracy = accuracy_score(y_train, rf_train_pred)
rf_test_accuracy = accuracy_score(y_test, rf_test_pred)
print(f'Random Forest Train Accuracy: {rf_train_accuracy:.4f}')
print(f'Random Forest Test Accuracy: {rf_test_accuracy:.4f}')

Random Forest Train Accuracy: 0.9790
Random Forest Test Accuracy: 0.9773


In [9]:
# 피처 중요도 확인
rf_feature_importances = rf.feature_importances_

In [10]:
# 피처 중요도를 데이터프레임으로 변환
feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': rf_feature_importances})

In [11]:
# 중요도 순으로 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [12]:
# 랜덤포레스트 피처 중요도 확인 및 저장
total_importance = sum(rf_feature_importances)
rf_feature_importance_percentages = (rf_feature_importances / total_importance) * 100
rf_feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': rf_feature_importance_percentages})
rf_feature_importance_df = rf_feature_importance_df.sort_values(by='Importance', ascending=False)
rf_feature_importance_df.to_excel('Overdue_RandomForest_feature_importance.xlsx', index=False)

### Test_data.xlsx CB예측

In [13]:
# test.xlsx 파일 로드
test_data = pd.read_excel('test_data.xlsx')

In [14]:
# test_data에 동일한 전처리 수행
test_data = pd.get_dummies(test_data, columns=['AD_NO', 'LIV_ADD', 'RES_ADD', 'ADD_YN'])

In [15]:
# 훈련 데이터와 동일한 특성을 가지도록 조정
missing_cols = set(X.columns) - set(test_data.columns)
for c in missing_cols:
    test_data[c] = 0
test_data = test_data[X.columns]

In [16]:
# 스케일링
test_data_scaled = scaler.transform(test_data)

In [17]:
# 훈련된 모델을 사용하여 예측 수행
test_pred = rf.predict(test_data_scaled)

In [18]:
# 예측 결과를 데이터프레임으로 변환
test_pred_df = pd.DataFrame(test_pred, columns=['연체 예측'])

In [19]:
# 결과를 엑셀 파일로 저장
test_pred_df.to_excel('test_RandomForest_Overdue.xlsx', index=False)

In [15]:
import joblib
joblib.dump(rf, 'Overdue_RandomForest_model.pkl')
joblib.dump(scaler, 'Overdue_RandomForest_scaler.pkl')

['Overdue_RandomForest_scaler.pkl']

In [16]:
cb_rf = joblib.load('CB_RandomForest_model.pkl')
cb_scaler = joblib.load('CB_RandomForest_scaler.pkl')